In [1]:
#stunde ask physics
# how does a magnet work
# explain feynman diagram

In [4]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms. 
You assume no prior knowledge.\n\nHere is the question: {query}'''

expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Physics.\n\nHere is the question: {query}'''

In [5]:
# route promopt infromation

prompt_infos = [
    {'name': 'beginner physics',
     'description': 'answer basic physics questions', 
     'template': beginner_template},
    {'name': 'advanced physics',
     'description': 'answers advanced physics questions',
     'template': expert_template}
]

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


In [42]:
llm = ChatOpenAI(model="gpt-4o-mini")

destination_chains = {}
for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = prompt | llm
    # chain = LLMChain(llm=llm, prompt=prompt)

    destination_chains[name] = chain

In [43]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = default_prompt | llm

In [44]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [45]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [46]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations

['beginner physics: answer basic physics questions',
 'advanced physics: answers advanced physics questions']

In [47]:
destination_str = "\n".join(destinations)
destination_str

'beginner physics: answer basic physics questions\nadvanced physics: answers advanced physics questions'

In [48]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [49]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)
print(router_template )

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner physics: answer basic physics questions
advanced physics: answers advanced physics questions

In [50]:
router_prompt = PromptTemplate(template=router_template,
                               input_variables=['input'],
                               output_parser=RouterOutputParser())

In [51]:
from langchain.chains.router import MultiPromptChain

In [52]:
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [53]:
destination_chains

{'beginner physics': ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are a physics teacher who is really\nfocused on beginners and explaining complex topics in simple to understand terms. \nYou assume no prior knowledge.\n\nHere is the question: {input}'), additional_kwargs={})])
 | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001BBD3458C50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001BBD302FF50>, root_client=<openai.OpenAI object at 0x000001BBD3459190>, root_async_client=<openai.AsyncOpenAI object at 0x000001BBD34594F0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')),
 'advanced physics': ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTempla

In [54]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True)

AttributeError: 'RunnableSequence' object has no attribute 'get'

In [55]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [71]:
beginner_template = '''You are a physics teacher who is really
focused on beginners and explaining complex topics in simple to understand terms. 
You assume no prior knowledge and do not give excessively long answers.\n\nHere is the question: {query}'''

expert_template = '''You are a world expert physics professor who explains physics topics
to advanced audience members. You can assume anyone you answer has a 
PhD level understanding of Physics.\n\nHere is the question: {query}'''

In [72]:
destination_chains

beginner_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', beginner_template),
        ('human', '{query}')
    ]
)

expert_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', expert_template),
        ('human', '{query}')
    ]
)

beginner_chain = beginner_prompt | llm | StrOutputParser()
expert_chain = expert_prompt | llm | StrOutputParser()


In [75]:
route_system = "Route the user's query to either the beginner or expert physics model."
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{query}"),
    ]
)

In [110]:
class RouteQuery(TypedDict):
    """Route query to destination."""
    destination: Literal["beginner", "expert"]


route_chain = (
    route_prompt
    | llm.with_structured_output(RouteQuery)
    | itemgetter("destination")
)

In [111]:
RouteQuery()

{}

In [112]:
route_chain.invoke(input={'query': 'explain Feynman diagrams'})

'expert'

In [108]:
chain = {
    "destination": route_chain,
    "query": lambda x: x["query"],
} | RunnableLambda(
    lambda x: beginner_chain if x['destination'] == 'beginner' else expert_chain,
) 


In [109]:
print(chain)

first={
  destination: ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="Route the user's query to either the beginner or expert physics model."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])
               | RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001BBD345F170>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001BBD3461520>, root_client=<openai.OpenAI object at 0x000001BBD302DC10>, root_async_client=<openai.AsyncOpenAI object at 0x000001BBD345F4A0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'RouteQ

In [104]:
print(chain.invoke({"query": "how does magnets work?"}))

AttributeError: 'dict' object has no attribute 'invoke'

In [95]:
print(chain.invoke({"query": "explain feynman diagrams?"}))

Feynman diagrams are a powerful pictorial representation of the interactions between particles in quantum field theory, particularly in the context of quantum electrodynamics (QED) and other gauge theories. They serve as a convenient tool for visualizing and calculating the probabilities of various scattering processes and decay events.

At a fundamental level, Feynman diagrams provide a way to encapsulate the perturbative expansion of the scattering amplitude in terms of simple graphical elements. Each element in a Feynman diagram corresponds to a specific mathematical term in the perturbation series.

**Key Components of Feynman Diagrams:**

1. **Lines:**
   - **Internal Lines:** Represent virtual particles that mediate interactions between real particles. These lines correspond to internal propagators in the mathematical formulation, typically expressed via Green's functions.
   - **External Lines:** Represent incoming and outgoing real particles in the scattering process. These lin

In [74]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner physics: answer basic physics questions
advanced physics: answers advanced physics questions

In [66]:
beginner_chain.invoke({"query": "how do magnets work?"})

AIMessage(content='Great question! Let’s break it down in a simple way.\n\n### What Are Magnets?\n\nMagnets are objects that can attract or repel other magnets and certain materials, like iron. They have two ends, called **poles**: a **north pole** and a **south pole**. \n\n### How Do They Work?\n\n1. **Magnetic Fields**: Magnets create an invisible area around them called a **magnetic field**. This field is what allows them to attract or repel other materials. You can think of it like a force field that extends around the magnet.\n\n2. **Atoms and Electrons**: The reason magnets work has to do with the tiny particles that make up everything around us, called **atoms**. Inside atoms, there are even smaller particles called **electrons**. Electrons can move and spin, and when they do, they create a tiny magnetic field.\n\n3. **Alignment**: In most materials, the magnetic fields of the electrons are all jumbled up, and they cancel each other out. However, in magnets, many of these tiny m

In [68]:
beginner_chain.invoke({"query": "how do magnets work?"})

'Great question! Let’s break it down simply.\n\n**What Are Magnets?**\nMagnets are objects that can attract certain types of materials, like iron, nickel, and cobalt. They have two ends called "poles" – one is the North Pole and the other is the South Pole.\n\n**How Do They Work?**\nAt the most basic level, magnets work because of the arrangement of tiny particles called atoms. Inside each atom, there are even smaller particles called electrons that move around. These electrons create tiny magnetic fields. \n\nIn most materials, these tiny magnetic fields point in different directions, so they cancel each other out, and the material doesn\'t act like a magnet. However, in magnetic materials, many of these tiny magnetic fields can line up in the same direction. When enough of them line up, the material itself becomes a magnet!\n\n**The Magnetic Field:**\nEvery magnet creates an invisible area around it called a magnetic field. This is the space where the magnet can exert a force on othe

In [69]:
print(beginner_chain.invoke({"query": "how do magnets work?"}))

Great question! Magnets are fascinating objects, and understanding how they work can be quite simple. Let’s break it down step by step.

### What is a Magnet?

A magnet is a material that can produce a magnetic field. This magnetic field is what allows magnets to attract or repel certain materials, like iron, cobalt, and nickel.

### The Basics of Magnetism

1. **Magnetic Poles**: Every magnet has two ends called poles: a **north pole** and a **south pole**. If you’ve ever played with magnets, you might have noticed that when you try to push together two north poles or two south poles, they repel each other (push away). But if you bring a north pole near a south pole, they attract (pull together). This is a key property of magnets!

2. **Magnetic Field**: A magnetic field is an invisible area around a magnet where magnetic forces can be felt. You can think of it like a bubble around the magnet. The strength of this field decreases with distance; the closer you are to the magnet, the st